In [1]:
import keyring
import os
OPENAI_API_KEY = keyring.get_password('openai', 'key_for_windows')
# https://tavily.com/   # search result
TAVILY_API_KEY = keyring.get_password('tavily', 'key_for_windows')
os.environ['OPENAI_API_KEY'] = OPENAI_API_KEY
os.environ['TAVILY_API_KEY'] = TAVILY_API_KEY


# Set State

In [2]:
from typing import Annotated
from typing_extensions import TypedDict
from langgraph.graph.message import add_messages

class State(TypedDict):
    messages: Annotated[list, add_messages]

# ToolNode로 도구 노드 구축

In [3]:
from langchain_community.tools.tavily_search import TavilySearchResults
from langgraph.prebuilt import ToolNode, tools_condition

tool = TavilySearchResults(max_results=2)
tools = [tool]
tool_node = ToolNode(tools)

# ToolNode가 무엇일까?

In [4]:
# # ToolNode Class

# import json

# from langchain_core.messages import ToolMessage

# class BasicToolNode:
#     """A node that runs the tools requested in the last AIMessage."""
    
#     def __init__(self, tools: list) -> None:
#         self.tools_by_name = {tool.name: tool for tool in tools}
        
#     def __call__(self, inputs: dict):
#         if messages := inputs.get("messages", []):
#             message = messages[-1]
#         else:
#             raise ValueError("No message found in input")
#         outputs = []
#         for tool_call in message.tool_calls:
#             tool_result = self.tools_by_name[tool_call["name"]].invoke(
#                 tool_call["args"]
#             )
#             outputs.append(
#                 ToolMessage(
#                     content=json.dumps(tool_result),
#                     name=tool_call["name"],
#                     tool_call_id=tool_call['id'],
#                 )
#             )
#         return {"messages": outputs}

# LLM 챗봇 설정

In [ ]:
from langchain_openai import ChatOpenAI

llm = ChatOpenAI(model='gpt-4o-mini')
llm_with_tools = llm.bind_tools(tools)
-
def chatbot(state: State):
    result = llm_with_tools.invoke(state['messages'])
    return {"messages": [result]}

# Build graphs

In [ ]:
from langgraph.graph import StateGraph

graph_builder = StateGraph(State)

graph_builder.add_node("chatbot", chatbot)
graph_builder.add_node("tools", tool_node)

graph_builder.add_edge("tools", "chatbot")
graph_builder.add_conditional_edges("chatbot", tools_condition) # tool 호출 여부를 판단할 수 있는 라이브러리 제공

graph_builder.set_entry_point("chatbot")
graph = graph_builder.compile()

## 인터넷 검색이 필요한 질문

In [7]:
graph.invoke({"messages": {"role": "user", "content": "2024년 한국 대통령은 누구야?"}})

{'messages': [HumanMessage(content='2024년 한국 대통령은 누구야?', additional_kwargs={}, response_metadata={}, id='5c1ce9b7-ea80-4498-b490-1fb873f511cc'),
  AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'call_9xYrpjoRumbc333g9AjkmZVO', 'function': {'arguments': '{"query":"2024년 한국 대통령"}', 'name': 'tavily_search_results_json'}, 'type': 'function'}], 'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 22, 'prompt_tokens': 88, 'total_tokens': 110, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_0705bf87c0', 'finish_reason': 'tool_calls', 'logprobs': None}, id='run-6246f7b2-e96c-405e-92ee-e012ff928875-0', tool_calls=[{'name': 'tavily_search_results_json', 'args': {'query': '2024년 한국 대통령'}, 'id': 'call_9xYrpjoRumbc333g9AjkmZVO', 'type': 'tool

## LLM이 답할 수 있는 질문

In [8]:
graph.invoke({"messages": {"role": "user", "content": "마이크로소프트가 어떤 회사야?"}})

{'messages': [HumanMessage(content='마이크로소프트가 어떤 회사야?', additional_kwargs={}, response_metadata={}, id='a36192c1-c807-4f45-bd17-297784bfdce6'),
  AIMessage(content='마이크로소프트(Microsoft)는 미국의 다국적 기술 회사로, 소프트웨어, 하드웨어, 전자기기 및 서비스 분야에서 활동하고 있습니다. 1975년 빌 게이츠와 폴 앨런에 의해 설립되었으며, 본사는 워싱턴주 레드먼드에 위치하고 있습니다.\n\n주요 제품으로는 다음과 같은 것들이 있습니다:\n\n1. **운영 체제**: Windows는 개인용 컴퓨터와 서버에서 널리 사용되는 운영 체제입니다.\n2. **오피스 제품군**: Microsoft Office는 워드, 엑셀, 파워포인트 등의 생산성 소프트웨어를 포함합니다.\n3. **클라우드 서비스**: Azure는 클라우드 컴퓨팅 플랫폼으로, 기업들이 클라우드에서 애플리케이션 및 서비스를 구축하고 관리할 수 있도록 지원합니다.\n4. **게임**: Xbox는 마이크로소프트의 게임 콘솔 브랜드이며, Xbox Live는 온라인 멀티플레이어 게임 서비스입니다.\n5. **소프트웨어 개발**: Visual Studio와 같은 개발 도구를 제공하여 개발자들이 소프트웨어를 만들고 배포할 수 있도록 돕습니다.\n\n마이크로소프트는 기술 혁신과 사회적 책임을 강조하며, 다양한 분야에 걸쳐 솔루션을 제공하고 있습니다.', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 293, 'prompt_tokens': 89, 'total_tokens': 382, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 